In [1]:
from flood_prediction.interface.main import *
%load_ext autoreload
%autoreload 2

2023-09-08 17:42:23.760784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 17:42:23.999218: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-09-08 17:42:23.999290: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-09-08 17:42:24.050080: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-08 17:42:25.377893: W tensorflow/stream_executor/platform/de

In [17]:
X_pred = api_request_pred()

Getting Weather Data
(336, 10)
Getting Flood Data
(336, 1)
Merging both dfs
(336, 12)


In [18]:
X_pred.columns

Index(['date', 'T(degC)', 'rain(mm)', 'surf_press(hPa)', 'wind_s(km/h)',
       'wind_dir(deg)', 'soil_moisture_0_1cm(m3)', 'soil_moisture_1_3cm(m3)',
       'soil_moisture_3_9cm(m3)', 'soil_moisture_9_27cm(m3)',
       'radiation(W/m2)', 'river_discharge(m3/s)'],
      dtype='object')

In [24]:
# printing date and "river_discharge(m3/s)" column every 24hs
X_pred[['date','river_discharge(m3/s)']].iloc[::24]    

,date,river_discharge(m3/s)
0,2023-08-26,110.17
24,2023-08-27,78.80
48,2023-08-28,64.01
72,2023-08-29,55.62
96,2023-08-30,49.58
120,2023-08-31,43.87
144,2023-09-01,40.11
168,2023-09-02,37.02
192,2023-09-03,37.37
216,2023-09-04,37.28


In [25]:
X_processed = preprocess_features_pred(X_pred)
X_processed.head()

Pre processing features pred
   T(degC)  rain(mm)  surf_press(hPa)  radiation(W/m2)  river_discharge(m3/s)  \
0      5.8       0.0            988.2              0.0                 110.17   
1      5.2       0.0            988.2              0.0                 110.17   
2      4.5       0.0            987.9              0.0                 110.17   
3      3.5       0.0            988.1              0.0                 110.17   
4      2.9       0.0            987.6              0.0                 110.17   

   soil_moist_0_to_28cm(m3)        Wx        Wy       day_sin   day_cos  \
0                   0.38325 -3.644269  5.015905 -1.117296e-11  1.000000   
1                   0.38400 -2.863777  5.385980  2.588190e-01  0.965926   
2                   0.38375 -2.905539  5.702442  5.000000e-01  0.866025   
3                   0.38425 -2.535710  5.437847  7.071068e-01  0.707107   
4                   0.38450 -2.535710  5.437847  8.660254e-01  0.500000   

   month_sin  month_cos  year_sin

,T(degC),rain(mm),surf_press(hPa),radiation(W/m2),river_discharge(m3/s),soil_moist_0_to_28cm(m3),Wx,Wy,day_sin,day_cos,month_sin,month_cos,year_sin,year_cos
0,5.8,0.0,988.2,0.0,110.17,0.38325,-3.644269,5.015905,-1.117296e-11,1.000000,-0.966271,0.257526,-0.806379,-0.591400
1,5.2,0.0,988.2,0.0,110.17,0.38400,-2.863777,5.385980,2.588190e-01,0.965926,-0.964021,0.265827,-0.806802,-0.590821
2,4.5,0.0,987.9,0.0,110.17,0.38375,-2.905539,5.702442,5.000000e-01,0.866025,-0.961698,0.274109,-0.807226,-0.590243
3,3.5,0.0,988.1,0.0,110.17,0.38425,-2.535710,5.437847,7.071068e-01,0.707107,-0.959305,0.282371,-0.807648,-0.589664
4,2.9,0.0,987.6,0.0,110.17,0.38450,-2.535710,5.437847,8.660254e-01,0.500000,-0.956841,0.290612,-0.808071,-0.589085


In [27]:
X_processed = tf.expand_dims(X_processed, axis=0)
X_processed

<tf.Tensor: shape=(1, 1, 336, 14), dtype=float64, numpy=
array([[[[ 5.80000000e+00,  0.00000000e+00,  9.88200000e+02, ...,
           2.57525834e-01, -8.06378639e-01, -5.91399603e-01],
         [ 5.20000000e+00,  0.00000000e+00,  9.88200000e+02, ...,
           2.65827481e-01, -8.06802337e-01, -5.90821453e-01],
         [ 4.50000000e+00,  0.00000000e+00,  9.87900000e+02, ...,
           2.74109461e-01, -8.07225620e-01, -5.90243000e-01],
         ...,
         [ 1.27000000e+01,  0.00000000e+00,  9.84400000e+02, ...,
           1.68686732e-02, -9.23340647e-01, -3.83981834e-01],
         [ 1.18000000e+01,  0.00000000e+00,  9.84500000e+02, ...,
           8.26799053e-03, -9.23615642e-01, -3.83319901e-01],
         [ 1.12000000e+01,  0.00000000e+00,  9.84800000e+02, ...,
          -3.33303800e-04, -9.23890161e-01, -3.82657771e-01]]]])>

In [ ]:
# printing df correlation matrix in blue colors
corr = X_pred.corr()
corr.style.background_gradient(cmap='coolwarm')